In [1]:
import h3
import folium
import numpy as np

from pathfinder.algo import (
    OutwardSpiralPathFinder, 
    BayesianHexSearch
)
from utils.hex import (
    hex_to_binary, 
    octal_list_to_binary, 
    binary_to_octal_list, 
    binary_to_hex, 
    hex_to_array_index,
    array_index_to_hex,
    distance_between_2_hexas,
)
from utils.viz import (
    gradient_color,
    add_hex_to_map,
    visualise_hex_dict_to_map
)
from experiment.test_framework import TestFramework

In [9]:
CENTRE = (1.340640225800291, 103.962424829341)
RES = 15
STEPS = 500
f = 0.99

test_spiral = TestFramework(name="Spiral", centre=CENTRE, res=RES, f=f)
test_spiral.register_pathfinder(OutwardSpiralPathFinder)
spiral_output = test_spiral.run(STEPS)
test_spiral.evaluate()

# test_bayes = TestFramework(name="Bayes", centre=CENTRE, res=RES, f=f)
# test_bayes.register_pathfinder(BayesianHexSearch)
# bayes_output = test_bayes.run(STEPS, update_map=True)
# test_bayes.evaluate()

Spiral's Path Coverage: 100.0%


In [10]:
m = folium.Map(location=[CENTRE[0],CENTRE[1]], zoom_start=23, max_zoom=25)

visualise_hex_dict_to_map(test_spiral.probability_map,m)
add_hex_to_map(spiral_output, m)

# visualise_hex_dict_to_map(test_bayes.probability_map,m)
# add_hex_to_map(bayes_output, m)


In [11]:
display(m)